# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: **QLoRA**
* Model: **BERT**
* Evaluation approach: **accuracy**
* Fine-tuning dataset: **GLUE/SST2**

In [1]:
# !pip install -q bitsandbytes peft transformers datasets evaluate scikit-learn

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [2]:

import shutil
import os
import torch
import math
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
import evaluate

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Use Base model BERT and tokenizer - suitable for sequence classification
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
# Use GLUE/SST2 dataset
dataset = load_dataset("glue", "sst2")

def preprocess(example):
    return tokenizer(example['sentence'], truncation=True, padding='max_length', max_length=128)

encoded_dataset = dataset.map(preprocess, batched=True)
encoded_dataset = encoded_dataset.rename_column("label", "labels")
encoded_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [6]:
# Use 'accuracy' as Evaluation Function
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=1).to(device)
    labels = torch.tensor(labels).to(device)
    return metric.compute(predictions=predictions, references=labels)

In [7]:
# Define training arguments for original model evaluation
eval_args = TrainingArguments(
    output_dir="./bert-sst2-eval",  # Temporary output directory for evaluation
    per_device_eval_batch_size=32,
    do_train=False,  # Do not train
    do_eval=True,    # Only evaluate
    remove_unused_columns=False, # Keep all columns for evaluation
)

# Load the original model WITHOUT quantization for initial evaluation
original_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
)

# Delete the folder if it exists
if os.path.exists(eval_args.output_dir):
    shutil.rmtree(eval_args.output_dir)
    print(f"Deleted existing directory: {eval_args.output_dir}")
    
# Initialize Trainer for original model evaluation
original_trainer = Trainer(
    model=original_model,
    args=eval_args, # Use eval_args for evaluation
    eval_dataset=encoded_dataset["validation"],
    compute_metrics=compute_metrics,
)

print("Original model evaluation:")
original_results = original_trainer.evaluate()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Deleted existing directory: ./bert-sst2-eval
Original model evaluation:


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [8]:
from peft import get_peft_model, LoraConfig, TaskType

# Configure 4-bit quantization for fine-tuning
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

In [9]:
# Load the model AGAIN with quantization for PEFT fine-tuning
quantized_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    quantization_config=bnb_config,
)

# Prepare the quantized model for k-bit training (gradient preparation)
quantized_model = prepare_model_for_kbit_training(quantized_model)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# configure LoRA 
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    target_modules=["query", "value"]
)

peft_model = get_peft_model(quantized_model, lora_config)

In [11]:
# Configure training args
training_args = TrainingArguments(
    output_dir="./bert-sst2-peft-qlora-eval",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    learning_rate=5e-4
)

# Delete the folder if it exists
if os.path.exists(training_args.output_dir):
    shutil.rmtree(training_args.output_dir)
    print(f"Deleted existing directory: {training_args.output_dir}")
    
# Init the trainer on the PEFT model
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=encoded_dataset["train"].shuffle(seed=42).select(range(1000)),  # small subset for quick tuning
    eval_dataset=encoded_dataset["validation"],
    compute_metrics=compute_metrics,
)

Deleted existing directory: ./bert-sst2-peft-qlora-eval


In [12]:
# Train...
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.614300,0.458650,0.819954
2,0.395800,0.326462,0.860092
3,0.279200,0.330601,0.858945


TrainOutput(global_step=189, training_loss=0.4488196196379485, metrics={'train_runtime': 83.348, 'train_samples_per_second': 35.994, 'train_steps_per_second': 2.268, 'total_flos': 198016312320000.0, 'train_loss': 0.4488196196379485, 'epoch': 3.0})

In [13]:
# Save the PEFT model 
path = './PEFT-Model'
if os.path.exists(path):
    shutil.rmtree(path)
    print(f"Deleted existing directory: {path}")
peft_model.save_pretrained(path)

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [14]:
# Load the saved PEFT model
from peft import AutoPeftModelForSequenceClassification
LoRA_model1_uploaded = AutoPeftModelForSequenceClassification.from_pretrained(path)
trainer.model = LoRA_model1_uploaded
trainer.model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(
                    in_features=768, out_features=768, bias=True
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_

In [15]:
# Evaluate the fine-tuned model
print("Fine-tuned model evaluation:")
fine_tuned_results = trainer.evaluate()

Fine-tuned model evaluation:


In [16]:
# Comparison original vs fine-tuned
print("BERT Accuracy:", original_results["eval_accuracy"])
print("Fine-Tuned BERT Accuracy:", fine_tuned_results["eval_accuracy"])

BERT Accuracy: 0.5137614678899083
Fine-Tuned BERT Accuracy: 0.8577981651376146
